In [ ]:
# Training

from ultralytics import YOLO

model = YOLO('./model/yolo.pt') 

results = model.train(
    data='./data/yolo/dataset.yaml', 
    epochs=100,           
    imgsz=500,
    batch=16,                
    patience=50,    
    device=0,  
)

metrics = model.val()

model.export(format='onnx')  # Exportiere in ONNX-Format

In [ ]:
# Visualization of model

img = "./data/segments/testing/front_floor_01_partial_room_17_pano_57.jpg" # validation image

results = model([img]) 

for result in results:
    boxes = result.boxes 
    masks = result.masks
    keypoints = result.keypoints 
    probs = result.probs 
    obb = result.obb
    result.show()